In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

#We may want these at some point for transforming our output:
#from scipy.special import logit, expit

pd.set_option('display.max_columns', None) #forces all columns to be displayed
pd.set_option('display.max_rows', None) #forces all rows to be displayed

In [4]:
filepath = r'../data/data_reduced_train.csv'
data = pd.read_csv(filepath)

In [7]:
xgb_parameters = {}
n_neighbors = 10 #neighbors for kNN imputation
percent_black_threshold = 0.5
percent_white_threshold = 0.5
percent_asian_threshold = 0.5
percent_hispanic_threshold = 0.5